# Table des Matières
* [Nombres](#nombres)
   - [Nombres aléatoires](#aléatoires)
   - [Précision des calculs](#précision)
* [Listes, Séquences et Dictonnaires](#listsseq)
   - [Listes](#lists)
   - [Dictionnaires](#hashes)
   - [Séqences](#sequences)
   - [Générateurs](#generators)
* [Fonctions](#functions)
   - [Gradual Typing](#gradual)
   - [Subsets](#subsets)
   - [Multi-Methods](#methods)
* [Concurrency](#concurrency)
   - [Parallel Processing](#parallel)
   - [Hyper Operators](#hyperoperators)
   - [Hyper Sequences](#hyperseqs)
   - [Asynchrone](#promises)
* [Grammars](#grammars)   


<a name="introduction"></a>
# Introduction

Raku, anciennement perl6, est un nouveau langage qui n'est PAS du tout compatible avec perl5, mais en garde néanmoins des aires de famille. C'est un langage entièrement object, donc toute (sauf exception) est un objet, et on peut appeler des methodes sur n'importe quoi. Le compilateur peut produire du code diriigé vers sa propre VM: MoarVM, ou le JVM, ou Javascript. Raku est enitèrement en Unicode, on peut donc utiliser toutes les caractères unicode, comme par exemple π, même en tant qu'opérateurs ou noms de variables et fonctions. 

Remerciements à toute la communauté Raku qui a fait les présentations et documents qui m'ont aidés à construire cette présentation. 

<a name="nombres"> </a>
# Nombres

<a name="aléatoires"> </a>
## Nombres aléatoires

C'est un problème généralisé, depuis toujours. Est-ce que votre langage de programmation as un mot clé `rand`, qui donne une réponse entre 0 et presque 1, ou une fonction `rand()`, qui donne une réponse entre 1 et l'argument?  Est-ce que c'est en point flottant? Est-ce qu'il faut l'arrondir? le tronquer? 


Raku fournit une fonction `roll` qui sélectionne au hasard un élément d'une liste (plutôt que de générer un entier)

In [ ]:
(1000..9999).roll;

Un exemple du genre d'utilisation: voici un dungeon master qui veut donner des attributs au hasrd à un joueur: 

In [ ]:
my @attributes = <Str Int Dex Con Cha>;

my @d6 = 1..6;

for @attributes -> $atr {
    my $stat = @d6.roll(3).sum;

    say "$atr: $stat"; 
}

Mais peut-être un dungeon master plus généreux prendrait les meilleurs 3 dés sur 5:

In [ ]:
my @attributes = <Str Int Dex Con Cha>;

my @d6 = 1..6;

for @attributes -> $atr {
    my $stat = @d6
        .roll(5)
        .sort(-*)
        .head(3).sum;
        
    say "$atr: $stat"; 
}

Enfin, `roll` permet la duplication, fait la sélection avec des doubles. Mais si ont veut avoir une sélection sans doubles? 

In [ ]:
my @friends = <Bob Martial Jean Freddie Bob Simon Jean>;
say @friends.pick(3);

<a name="précision"></a> 
## Précision des calculs

Le point-flottant est un problème depuis toujours. Sur STack Overflow, il y a d'innombrables questions dur le poinmt flottant. 

En mathèmathiques, l'énoncé `0.3 = 0.1 + 0.2`  est vrai. Mais, en python? en ruby?

In [ ]:
%%bash
ruby -e 'puts 0.3 == 0.1 + 0.2'

In [ ]:
%%bash 
python -c 'print(0.1 + 0.2 == 0.3)'

Python, Perl5, C, et maintes autres langages donnent la même reponse. 

Mais, en Raku, la réponse est correcte!

In [ ]:
say 0.1 + 0.2 == 0.3 ?? "yes" !! "No" ;

In [ ]:
say 0.3.nude;

In [ ]:
say (1/12 + 1/5).nude;

In [ ]:
0.3.WHAT

Dans Haskell, ce sont des "Rationals". Dans Raku, ce sont des Rat. (Et si les chiffres deviennent assez frands, des BigRat)

C'est plus lent que du point-flottant `hardware`, mais ça donne la bonne réponse, ce que le point flottant ne fait pas. 

Un autre exemple: $7\over 3$

In [ ]:
say -7/3;
say (-7/3).nude;
say (-7/3).WHAT;

`pi` (`π`) et `tau` (τ) sont des built-in:

In [ ]:
say pi == π;
say pi;

say tau == τ;
say tau;
tau.WHAT;

<a name="listsseq"></a>
# Listes, séquences et dictionnaires

<a name="lists"></a>
## Listes

Des listes: c'est la virgule qui est l'opérateur de liste.

In [ ]:
say my @a = [ 1, 2, 3 ];

say @a = 1, 2, 3;

say @a = [ 'un', 'deux', 'trois' ];

say @a = < un deux trois >; #un raccourci pour des mots
say < un deux trois >.WHAT;

say @a.keys;

<a name='hashes'></a>
## Dictionnaires

Les dictionnaires sont appelés des `hash`. Voici quelques exemples de syntaxes possibles:

In [ ]:
say my $h = { 'un' => 1, 'deux' => 2 };
say {'un' => 1, 'deux' => 2 }.WHAT;

say $h = { un => 1, deux => 2 }; # s'il n'y apas d'espace dans la clé

say %h = < a b c d e f >;

say %h = :un<1>, :deux<2>;
(:un<1>, :deux<2> ).WHAT.say;

say %h = :un(1), :deux(2);

In [ ]:
say %h.keys;
say %h.values;
say %h{'un'};
say %h<deux>;

<a name="sequences"></a>
## Séquences

Des listes qui ne sont pas entiérement evaluées encore, donc des *lazy lists*. Ces listes sont evalués au fur et à mesure que les valeurs sont demandés.

In [ ]:
say 0..10;  # même chose que python 'range(10)'
say (0..10).eager;  # eager force l'évaluation immediate (.lazy ferait le contraire)

say (0..^10).eager;
say (^10).eager;

Le chapeau indique *non-inclus*.  Donc:

In [ ]:
say (0^..10).eager;
say (0^..^10).eager;

In [ ]:
say 5 == ^10;
say ^10 == 5;

Ce n'est pas limité aux numéros:

In [ ]:
say ('a'..'zz')[37];

<a name="generators"></a>
## Générateurs

Le `...` ou le `…` est l'opérateur générateur. ça génère une liste 

In [ ]:
say 1 ... 4;
say 4 ... 1;
say 0.1, 0.2 ... 0.5; # raku détermine correctement que le pas est de 0.1

say 'a' ... 'd';

Attention!! Une liste infini:

In [ ]:
my @à-l'infini = 0, 2 ... *;  # avance par 2
say @à-l'infini[1020];

@à-l'infini = 1, 2, 4  ... ∞;  # même chose, mais double à chaque pas
say @à-l'infini[^12];

Si on ajoute un block pour calculer la prochaine valeur, elle doit être le dernier item avant le `...`. 
Voici un feedback loop avec un amplificateur: $4x(1-x)$

In [ ]:
say ( 0.2, { 4*$_*(1-$_) } ... * )[^5];

In [ ]:
say ('a', 'b', * ~ * ... *)[^5];

In [ ]:
say 1, 2, { sleep 1; last if $_ > 5; ++$_ } ... 10;


### Palindromes

Un petit cadeau !!

In [ ]:
#say (^100).grep({ $_ eq $_.flip });
say (^100).grep({ $_ eq $_.flip }).elems;

<a name="functions"></a> 
# Fonctions

Des fonctions, methodes ou lambdas en raku sont des appelés des`subroutines`. Il y a maintes façons de les déclarer:

In [ ]:
sub my-func { say "Look ma, no args!" }
my-func;

On peut les placer dans n'importe quel contenant (ou pas)

In [ ]:
my &c = sub { say "Look ma, no name!" }
c;     # OUTPUT: «Look ma, no name!␤» 
 
my Any:D $f = sub { say 'Still nameless...' }  # retourne n'importe quoi (type Any)
$f();  # OUTPUT: «Still nameless...␤» 

my $nameless = -> { say 'also nameless' } # un `pointy block`
$nameless();
 
my Code \a = sub { say ‚raw containers don't implement postcircumfix:<( )>‘ };
a.();  # OUTPUT: «raw containers don't implement postcircumfix:<( )>␤»

Une fonction peut recevoir des arguments, séparés par des virgules, même avec des valeurs par défault:

In [ ]:
sub exclaim ( $phrase, $name = 'John' ) {
    say "$phrase $name !!!!";
}

exclaim( 'Salut, mon chum' );

Une methode est aussi une fonction. Comme on peut s';imaginer, c'ets une fonction avec un argument présumé, en raku on appele ça un `invocant`

In [ ]:
my @a = [];

# @a c'est l'invocant

@a.push(1);  # appel de methode
push(@a,1);  # appel de fonction avec invocant au premier argument 
push @a: 1;  # même chose (synytaxe 'liste' simplifié, le : identifie l'invocant)

<a name="gradual"></a>
## Gradual Typing

`Gradual typing` veut dire qu'on n'a pas besoin de spécifier des types d'arguments pour une fonction, mais si on les spécifie, ça permet de mieux gérer la compilation et l'exécution. Raku amène ça un peu plus loin que d'autres. 

L'ensemble d'arguments ainsi que la valeur de retour est la signature d'une fonction. 

In [ ]:
sub fmt( $lineno, Str $str ) {
    say "$lineno: $str";
}
    
fmt( 1, "This must be a string" );
#fmt 2, [ 1, 2, 3];
#fmt 3, π;

On peut aussi mettre des contraintes:

In [ ]:
    sub fmt( $lineno where * < 10, Str $str where *.chars == 5 ) {
        say "$lineno, $str";
    }
    
    #fmt( 1, "This must be a string" );
    fmt( 1, "afive");
    fmt(12,"afive");

<a name="subsets"></a>
## Subsets

Une autre façon de faire ce genre de chose est de déclarer des types avec contraintes:

In [ ]:
subset SmallStr of Str where *.chars < 10;

In [ ]:
sub baby-twitter( SmallStr $str ) {
   say "This string must be less that 10 chars: $str"
}

#baby-twitter 'a longer string';
baby-twitter 'tiny twit';

<a name="methods"></a>
## Multi-Methods

Plusieurs langages de programmation ont la capacité de définir plusieurs fois la même fonction ou methode, distingué par des arguments différents.
Raku amène ça à un autre niveau.

In [ ]:
subset Prime      of Int   where *.is-prime;
subset BigPrime   of Prime where * >  10_000;
subset SmallPrime of Prime where * <= 10_000;

In [ ]:
multi sub test-prime ( BigPrime   $num ) { say "Prime number! Nice and big"; }
multi sub test-prime ( SmallPrime $num ) { say "Puny prime number";          }
multi sub test-prime (            $num ) { say "Gimme primes!";              }

In [ ]:
test-prime 3;
test-prime(31337);
test-prime 100_000;

On peut même étendre des classes:

In [ ]:
class Numbers {
    multi method id ( Numeric $num ) { say "$num is a number"       }
    multi method id (         $num ) { say "$num is something else" }
}
 
class SmarterNumbers is Numbers {
    multi method id ( Numeric $num where * == π ) { say "Mmmm yummy pie!" }
}
 
SmarterNumbers.new.id: 42;
SmarterNumbers.new.id: π;
SmarterNumbers.new.id: 'blah';

<a name="whatevercode"></a>
## WhateverCode

*WhateverCode* est un genre de *closure* avec des paramètres automatiques....

In [ ]:
say ( * + 2 )(2);
say sub { $^a + 2 }(2);
say sub ( $a ) { $a + 2 }(2);

C'est pas mal ce que le nom dit: un n'importe-quoi qui fait exactement ce qu'on veut:

In [ ]:
# des fois, ça remplace le $_:
say < 1 25 3 100 >.grep: * > 5;

# subset Prime1 of Int where *.is-prime;
# subset Prime2 of Int where { $_.is-prime };

In [ ]:
# Des fois, ça sert à ramasser plusieurs arguments:
say ( * + * + * )( 2, 3, 4 ); 

In [ ]:
# Des fois, c'est un raccourci pour un map:  
say ^12 .map: * + * + *;  # un map trois éléments à la fois

Série Fibonacci:

In [ ]:
my @fibonacci = 0, 1, * + * ... *;
say @fibonacci[5, 8, 10, 37];

<a name=concurrency></a>
# Concurrency

<a name=parallel></a>
## Parallel Processing

Le traitement en parallel est souvent difficile à gérer, et très difficile à debugger. En fait, sans passer par des processus, la majorité des langages de scripting communs aujourd'hui ont un 'Global Interpreter Lock' qui fait en sorte que on ne peut pas rouler en multi-thread. Il faut alors se pencher sur des processus séparés, ce qui rends les choses plus difficiles encore. 

<a name="hyperoperators"></a>
## Hyper Operators

Imaginons que les deux vecteurs `@row1` et `@row2` ont des millions d'éléments. 

In [ ]:
my @row1 = 5, 20, 3;
my @row2 = 3, 2, 1;

# say row_add( @row1, @row2 );

Un premier essai très simple: 

In [ ]:
sub row_add( @a, @b ) {
    die if @a.elems != @b.elems;
    
    my @result;
    for 0 .. @a.end -> $i {
        push @result, @a[$i] + @b[$i];
        }
     return @result;   
}

C'est sériel, un pas après l'autre. 

In [ ]:
sub row_add( @a, @b ) {
    die if @a.elems != @b.elems;
    
    return map { @a[$_] + @b[$_] }, 0 .. @a.end;
}

La fonction `keys` fournit la liste d'indexes: 

In [ ]:
sub row_add( @a, @b ) {
    die if @a.elems != @b.elems;
    
    return map { @a[$_] + @b[$_] }, @a.keys;
}

L'opérateur Zip (semblable à la fonction zip en python):

In [ ]:
sub row_add( @a, @b ) {
    die if @2.elems != @b.elems;

    return @a Z+ @b;
}

Mais toutes ces méthodes sont en série. 

In [ ]:
sub row_add( @a, @b ) {
    return @a »+« @b; # les deux longeurs doivent être identiques (exception si pas égal)
    
    #return @a <<+<< @b; retourne resultat de taille de @b, @a est réutilisé
    #return @a >>+>> @b; retourne resultat de taille de @a, @b est réutilisé
    #return @a <<+>> @b; retourne un résultat de taille du plus grand vecteur, le plus court est réutilisé
    
}

In [ ]:
say row_add( @row1, @row2 );

Les  `hyper-operators` sont potentiellement multi-thread; ils ne forcent pas le multi-thread, mais disent au compilateur que il n'y a pas d'effets secondaires, donc il peut y aller si c'est possible.

In [ ]:
say ( 1, 2 ) «+« ( 3, 4, 5 );
say ( 1, 2 ) »+» 1;
say ( 1, 2, 3, 4 ) «*» ( 1, 2 );

Un exemple très utile c'est quand on fait du `scaling`:

In [ ]:
sub scale ( @a, $n ) { return @a »*» $n }

Une différence importante entre le meta-operateur `Z` et les hyper-opérateurs « et » est que (autres que le `multi-threading` est que `Z` ne fait qu'un niveau, tandis que « et » traversent complètement la structure. 

<a name="hyperseqs"></a>
## Hyper Sequences

In [ ]:
for (1..4).race( batch => 1 ) {
    say "Doing $_";
    sleep 1;
}
say "Code took {now - INIT now} seconds to run";

la methode `race` ne préserve pas l'ordre, `hyper` est identique mais préserve l'ordre. 

<a name="promises"></a>
## Asynchrone (i.e. Promises)

Expression asynchrone avec `start`/`await`:

In [ ]:
my @promises = ^3 .map: {
    start {
        .say; sleep 1;
        $_ * 4;
    }
};
say "Commencé! {now - INIT now}";
say await @promises;
say "Complet! {now - INIT now}";

Une autre variété via la class `Promise`:

In [ ]:
Promise.in(5).then: -> $v { say "Ça fait {now - INIT now} secondes!" };
sleep 7;
say "Durée total {now - INIT now} secondes"

<a name="grammars"></a>
# Grammars

Perl était reconnu pour ses regexs. MUltiples langages de programmation on copiés ou directement implantés les regexs de perl, même s'ils ne servaient pas autant. Mais les regexs sont difficile à comprendre, même très difficile à écrire moindrement que les textes deviennent compliqués. 

Prenons, par exemple, une date iso, em perl5 on ferait:

mais, si on a besoin de lire deux dates, on fait quoi?  Prenons un fichier avec des lignes tel que: `IsoDate IsoDate Titre`

In [ ]:
grammar LogLine {
    token year  { <.digit> ** 4 <?{ $/.Num >= 1900  }> }
    token month { <.digit> ** 2 <?{ $/.Num ~~ 1..12 }> }
    token day   { <.digit> ** 2 <?{ $/.Num ~~ 1..31 }> }
    
    token ISO-Date { <year> '-' <month> '-' <day> }
    regex book-name { \S .*? }
    token start-date { <ISO-Date> }
    token stop-date  { <ISO-Date> }
    rule TOP { ^ <start-date> <stop-date> <book-name> $ }
    }

In [ ]:
my $line = ' 2010-02-28 2015-03-14 Harry Potter and the Methods of Rationality';

LogLine.parse($line) or die;

say $/<book-name>.Str.perl;

`Grammar` crée un `recursive-descent` parser. Pour faciliter les choses, il y a deux outils `Grammar::Tracer` et `Grammar::Debugger` pour aider à vérfier le code. 